In [1]:
from google.colab import drive
from google.colab.patches import cv2_imshow
from torchvision import transforms
drive.mount(('/content/drive/'))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
import torch
import numpy as np
import torchvision
import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cv2
import matplotlib.pyplot as plt
import math

In [3]:
from glob import glob
import pandas

In [4]:
device = 'cuda:0'
num_joints = 21

In [5]:
import torch.nn as nn
import torch
import torch.nn.functional as F

device = 'cuda:0'


# 2d pose estimator - pretrained
class CPM2DPose(nn.Module):
    def __init__(self):
        super(CPM2DPose, self).__init__()
        
        self.scoremap_list = []
        self.layers_per_block = [2, 2, 4, 2]
        self.out_chan_list = [64, 128, 256, 512]
        self.pool_list = [True, True, True, False]

        self.relu = F.leaky_relu
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_1
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_4 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_7 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv5_1 = nn.Conv2d(128, 512, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv5_2 = nn.Conv2d(512, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv6_1 = nn.Conv2d(149, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_3 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_4 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_5 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_6 = nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv6_7 = nn.Conv2d(128, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv7_1 = nn.Conv2d(149, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_3 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_4 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_5 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_6 = nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv7_7 = nn.Conv2d(128, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc_layer = nn.Sequential( # image -> semantic class로 나오도록
            nn.Linear(21*32*32,256),
            nn.ReLU(),
            nn.Linear(256,3)
        )
        

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.conv1_1(x))
        x = self.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2_1(x))
        x = self.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.relu(self.conv3_3(x))
        x = self.relu(self.conv3_4(x))
        x = self.maxpool(x)
        x = self.relu(self.conv4_1(x))
        x = self.relu(self.conv4_2(x))
        x = self.relu(self.conv4_3(x))
        x = self.relu(self.conv4_4(x))
        x = self.relu(self.conv4_5(x))
        x = self.relu(self.conv4_6(x))
        encoding = self.relu(self.conv4_7(x))
        x = self.relu(self.conv5_1(encoding))
        scoremap = self.conv5_2(x)

        x = torch.cat([scoremap, encoding], 1)
        x = self.relu(self.conv6_1(x))
        x = self.relu(self.conv6_2(x))
        x = self.relu(self.conv6_3(x))
        x = self.relu(self.conv6_4(x))
        x = self.relu(self.conv6_5(x))
        x = self.relu(self.conv6_6(x))
        scoremap = self.conv6_7(x)
        x = torch.cat([scoremap, encoding], 1)
        x = self.relu(self.conv7_1(x))
        x = self.relu(self.conv7_2(x))
        x = self.relu(self.conv7_3(x))
        x = self.relu(self.conv7_4(x))
        x = self.relu(self.conv7_5(x))
        x = self.relu(self.conv7_6(x))
        x = self.conv7_7(x)
        x = self.fc_layer(x)
        return x


In [6]:
'''
class Classifier(nn.Module):
    def __init__(self, batch_size):
        super(Classifier, self).__init__()
        self.batch_size = batch_size

        self.fc_layer = nn.Sequential(
            nn.Linear(21*32*32,256),
            nn.ReLU(),
            nn.Linear(256,3)
        )
      
    def forward(self, x):
        x = x.view(self.batch_size, -1)
        x = self.fc_layer(x)
        return x
'''

'\nclass Classifier(nn.Module):\n    def __init__(self, batch_size):\n        super(Classifier, self).__init__()\n        self.batch_size = batch_size\n\n        self.fc_layer = nn.Sequential(\n            nn.Linear(21*32*32,256),\n            nn.ReLU(),\n            nn.Linear(256,3)\n        )\n      \n    def forward(self, x):\n        x = x.view(self.batch_size, -1)\n        x = self.fc_layer(x)\n        return x\n'

In [7]:
train_img_len = 200
test_img_len = 30

class HandDataset(Dataset):
    def __init__(self, method=None):
        self.x_data = []
        self.y_data = []
        self.z_data = []
        self.root = '/content/drive/My Drive/hand_posture_data'
        self.weight_root = self.root

        if method == 'train':
            self.root = self.root + '/train/' # self.root = drive/My Drive/hand_posture_data/train/
            rock_path = self.root + 'rock/'
            scissors_path = self.root + 'scissors/'
            paper_path = self.root + 'paper/'

            self.rock_img_path = sorted(glob(rock_path + 'rgb/*.jpg'))
            self.scissors_img_path = sorted(glob(scissors_path + 'rgb/*.jpg'))
            self.paper_img_path = sorted(glob(paper_path + 'rgb/*.jpg'))

            self.img_path = self.rock_img_path + self.scissors_img_path + self.paper_img_path

        elif method == 'test':
            self.root = self.root + '/test/' # self.root = drive/My Drive/hand_posture_data/test/
            rock_path = self.root + 'rock/'
            scissors_path = self.root + 'scissors/'
            paper_path = self.root + 'paper/'

            self.rock_img_path = sorted(glob(rock_path + 'rgb/*.jpg'))
            self.scissors_img_path = sorted(glob(scissors_path + 'rgb/*.jpg'))
            self.paper_img_path = sorted(glob(paper_path + 'rgb/*.jpg'))

            self.img_path = self.rock_img_path + self.scissors_img_path + self.paper_img_path

        for i in tqdm.tqdm(range(len(self.img_path))):
            img = cv2.imread(self.img_path[i], cv2.IMREAD_COLOR)
            print(self.img_path[i])  
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            self.x_data.append(img)

            num = self.img_path[i].split('.')[0].split('/')[-1]
            
            if method == 'train':
              if i in range (train_img_len):
                img_pkl = self.root + 'rock/meta/' + str(num) + '.pkl'
              elif i in range (1*train_img_len, 2*train_img_len):
                img_pkl = self.root + 'scissors/meta/' + str(num) + '.pkl'
              elif i in range(2*train_img_len, 3*train_img_len):
                img_pkl = self.root + 'paper/meta/' + str(num) + '.pkl'

            elif method == 'test':
              if i in range(test_img_len):
                img_pkl = self.root + 'rock/meta/' + str(num) + '.pkl'
              elif i in range(test_img_len, 2*test_img_len):
                img_pkl = self.root + 'scissors/meta/' + str(num) + '.pkl'
              elif i in range(2*test_img_len, 3*test_img_len):
                img_pkl = self.root + 'paper/meta/' + str(num) + '.pkl'

            pkl = pandas.read_pickle(img_pkl)
            coords_2d = pkl['coords_2d']
            # coords_2d의 shape = 21*2
            self.y_data.append(coords_2d)

        length = 0
        if method == 'train':
          length = train_img_len
        elif method == 'test':
          length = test_img_len

        for i in range(3):  # rock = 0, scissors = 1, paper = 2
          for j in range(length):
            self.z_data.append(i)

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        transform1 = torchvision.transforms.ToTensor()
        new_x_data = transform1(self.x_data[idx])

        return new_x_data, self.y_data[idx], self.z_data[idx]
    

In [8]:
class Trainer(object):
    def __init__(self, epochs, batch_size, lr):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = lr
        dataset = HandDataset(method='train')
        self.weight_root = dataset.weight_root # weight_root = drive/My Drive/hand_posture_data
        self._build_model()
        self.root = dataset.root # self.root = drive/My Drive/hand_posture_data/train/
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

        print("Training...")

    def _build_model(self):
        # 2d pose estimator
        poseNet = CPM2DPose()
        self.poseNet = poseNet.to(device)
        self.poseNet.train() # train 모드 명시
        self.poseNet.load_state_dict(torch.load(self.weight_root+'/pretrained_weight.pth'), strict=False)

        print('Finish build model.')

    def skeleton2heatmap(self, _heatmap, keypoint_targets):
        heatmap_gt = torch.zeros_like(_heatmap, device=_heatmap.device) 
        # _heatmap은 ground_truth heatmap만들때 dimension 같게 해주려고 참고하는 용도로만 사용

        keypoint_targets = (((keypoint_targets)) // 8)
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                x = int(keypoint_targets[i, j, 0])
                y = int(keypoint_targets[i, j, 1])
                heatmap_gt[i, j, x, y] = 1

        heatmap_gt = heatmap_gt.detach().cpu().numpy()
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                heatmap_gt[i, j, :, :] = cv2.GaussianBlur(heatmap_gt[i, j, :, :], ksize=(3, 3), sigmaX=2, sigmaY=2) * 9 / 1.1772
        heatmap_gt = torch.FloatTensor(heatmap_gt).to(device)
        return heatmap_gt # ground truth heatmap


    def heatmap2skeleton(self, heatmapsPoseNet):
        skeletons = np.zeros((heatmapsPoseNet.shape[0], heatmapsPoseNet.shape[1], 2))
        for m in range(heatmapsPoseNet.shape[0]):
            for i in range(heatmapsPoseNet.shape[1]):
                u, v = np.unravel_index(np.argmax(heatmapsPoseNet[m][i]), (32, 32))
                skeletons[m, i, 0] = u * 8
                skeletons[m, i, 1] = v * 8
        return skeletons


    def train(self):
        losses = []
        loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.poseNet.parameters(), self.learning_rate)

        for param in self.poseNet.parameters():
          param.required_grad = True
        
        for epoch in tqdm.tqdm(range(self.epochs + 1)):
            loss = 0
            for batch_idx, samples in enumerate(self.dataloader):
                optimizer.zero_grad()
                x_train, y_train, z_train = samples #z_train은 semantic class정보(rock=0/scissors=1/paper=2)

                pred = self.poseNet(x_train.cuda())

                loss = loss_func(pred, z_train)
                loss += loss.item()
                loss.backward()  # backpropagation
                optimizer.step() # update parameters

            print('training loss: ', loss)
            losses.append(loss)

            if epoch == self.epochs:
               torch.save(self.poseNet.state_dict(), self.weight_root+'/pretrained_weight.pth')
        

        plt.plot(losses, color = 'r')
        print('Finish training.')


In [9]:
class Tester(object):
    def __init__(self, batch_size):

        self.batch_size = batch_size
        dataset = HandDataset(method='test')
        self.CPM_weight_root = dataset.weight_root + '/pretrained_weight.pth'
        self._build_model()
        self.root = dataset.root # self.root = drive/My Drive/hand_posture_data/test/
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=2)

        self.datalen = dataset.__len__()

        print("Testing...")

    def _build_model(self):
        # 2d pose estimator
        poseNet = CPM2DPose()
        self.poseNet = poseNet.to(device)
        self.poseNet.load_state_dict(torch.load(self.CPM_weight_root))

    def skeleton2heatmap(self, _heatmap, keypoint_targets):
        heatmap_gt = torch.zeros_like(_heatmap, device=_heatmap.device) 

        keypoint_targets = (((keypoint_targets)) // 8)
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                x = int(keypoint_targets[i, j, 0])
                y = int(keypoint_targets[i, j, 1])
                heatmap_gt[i, j, x, y] = 1

        heatmap_gt = heatmap_gt.detach().cpu().numpy()
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                heatmap_gt[i, j, :, :] = cv2.GaussianBlur(heatmap_gt[i, j, :, :], ksize=(3, 3), sigmaX=2, sigmaY=2) * 9 / 1.1772
        heatmap_gt = torch.FloatTensor(heatmap_gt).to(device)
        return heatmap_gt 

    def test(self):
        correct = 0
        total = 0
        for batch_idx, samples in enumerate(self.dataloader): 
            x_test, y_test, z_test = samples 
            pred = self.poseNet(x_test.cuda())

            for k in range(self.batch_size):
              total +=1
              if np.argmax(pred[k].detach().cpu().numpy()) == z_test[k]:
                correct+=1
            
 
        print('Overall Accuracy: ', correct/total)
        self.overall_accuracy = correct/total



In [10]:
def main():
    
    epochs = 20
    batchSize = 10
    learningRate = 1e-2

    trainer = Trainer(epochs, batchSize, learningRate)
    trainer.train()

    tester = Tester(batchSize)
    tester.test()



if __name__ == '__main__':
    main()

  6%|▌         | 35/600 [00:00<00:03, 172.04it/s]

/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000000.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000001.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000002.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000003.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000004.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000005.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000006.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000007.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000008.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000009.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000010.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000011.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000012.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000013.jpg
/content/drive/My Dr

 12%|█▏        | 69/600 [00:00<00:03, 167.54it/s]

/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000035.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000036.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000037.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000038.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000039.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000040.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000041.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000042.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000043.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000044.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000045.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000046.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000047.jpg
/content/drive/My Drive/hand_posture_data/train/rock/rgb/00000048.jpg
/content/drive/My Dr

 17%|█▋        | 104/600 [00:00<00:02, 167.05it/s]

...

/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000185.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000186.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000187.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000188.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000189.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000190.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000191.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000192.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000193.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000194.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000195.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000196.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000197.jpg
/content/drive/My Drive/hand_posture_data/train/paper/rgb/00000198.jpg
/conte

  0%|          | 0/21 [00:00<?, ?it/s]

Finish build model.
Training...


  0%|          | 0/21 [00:00<?, ?it/s]


RuntimeError: ignored